# NiHSA flood record
<!-- markdownlint-disable MD013 -->

In [2]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [23]:
import pandas as pd

from src.datasources import nihsa, codab
from src.utils import blob
from src.constants import *

In [4]:
adm2 = codab.load_codab_from_blob(admin_level=2)

In [5]:
adm2

,ADM2_EN,ADM2_PCODE,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,DATE,VALIDON,VALIDTO,geometry
0,Aba North,NG001001,Abia,NG001,Nigeria,NG,2016-11-29,2019-04-17,None,"POLYGON ((7.40013 5.08237, 7.39849 5.08255, 7...."
1,Aba South,NG001002,Abia,NG001,Nigeria,NG,2016-11-29,2019-04-17,None,"POLYGON ((7.38632 5.08236, 7.38529 5.08194, 7...."
2,Abadam,NG008001,Borno,NG008,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((13.11029 13.10963, 13.10815 13.11404..."
3,Abaji,NG015001,Federal Capital Territory,NG015,Nigeria,NG,2016-11-29,2019-04-17,None,"POLYGON ((7.02654 9.22216, 7.02314 9.22214, 7...."
4,Abak,NG003001,Akwa Ibom,NG003,Nigeria,NG,2016-11-29,2019-04-17,None,"POLYGON ((7.81233 5.09367, 7.81465 5.09137, 7...."
...,...,...,...,...,...,...,...,...,...,...
769,Zango-Kataf,NG019022,Kaduna,NG019,Nigeria,NG,2016-11-29,2019-04-17,None,"POLYGON ((8.51729 9.73228, 8.51795 9.72981, 8...."
770,Zaria,NG019023,Kaduna,NG019,Nigeria,NG,2016-11-29,2019-04-17,None,"POLYGON ((7.77156 11.11693, 7.76950 11.11178, ..."
771,Zing,NG035016,Taraba,NG035,Nigeria,NG,2016-11-29,2019-04-17,None,"POLYGON ((11.76143 8.66650, 11.76046 8.66366, ..."
772,Zurmi,NG037014,Zamfara,NG037,Nigeria,NG,2016-11-29,2019-04-17,None,"POLYGON ((7.04995 12.97205, 7.04315 12.94255, ..."


In [6]:
blob_name = f"{blob.PROJECT_PREFIX}/raw/AA-nigeria_data/NiHSA/LGA_Flood_History 2013-2023.xlsx"
dfs = blob.load_excel_from_blob(blob_name, sheet_name=None)

In [41]:
df_nihsa = (
    pd.concat(dfs.values(), keys=dfs.keys(), names=["Year", "Index"])
    .reset_index()
    .drop(columns="Index")
)
df_nihsa["LGA"] = df_nihsa["LGA"].fillna(df_nihsa["LGAs"])
df_nihsa["State"] = df_nihsa["State"].fillna(df_nihsa["STATE"])
df_nihsa = df_nihsa.drop(columns=["LGAs", "STATE"])
df_nihsa["Flooded"] = df_nihsa["Flooded"].astype(bool)
df_nihsa["Year"] = df_nihsa["Year"].astype(int)
df_nihsa = df_nihsa.dropna()
df_nihsa = df_nihsa[~df_nihsa["LGA"].isin(["Lake chad", "Lagos Lagoon"])]
df_nihsa["LGA"] = df_nihsa["LGA"].replace(
    {
        "Bukuru": "Buruku",
        "Koton-Karfe": "Kogi",
        "Girie": "Girei",
        "Teungo": "Toungo",
        "Borsari": "Bursari",
        "Barde": "Bade",
    }
)

In [42]:
df_nihsa = df_nihsa[df_nihsa["State"].isin(["Adamawa", "Borno", "Yobe"])]

In [43]:
df_nihsa

,Year,State,LGA,Flooded
17,2013,Adamawa,Demsa,False
18,2013,Adamawa,Fufore,False
19,2013,Adamawa,Ganye,False
20,2013,Adamawa,Girei,False
21,2013,Adamawa,Gombi,False
...,...,...,...,...
8507,2023,Yobe,Bursari,False
8508,2023,Yobe,Karasuwa,False
8509,2023,Yobe,Yusufari,False
8510,2023,Yobe,Yunusari,True


In [44]:
import pandas as pd
from thefuzz import process

# Example dataframes: df_nihsa and adm2
# df_nihsa['LGA'] should match adm2['ADM2_EN']


def match_LGAs(df_nihsa, adm2, threshold=90):
    # Create an empty dictionary to store the matched values
    matches = {}

    # Iterate over each LGA in df_nihsa
    for lga in df_nihsa["LGA"]:
        # Use thefuzz to find the best match in the 'ADM2_EN' column of adm2
        result = process.extractOne(lga, adm2["ADM2_EN"])

        # Print the result to inspect its structure (for debugging)
        # print(f"Result for '{lga}': {result}")

        # Check if the result is a tuple or dictionary
        if isinstance(result, tuple):
            best_match, score, _ = result
        elif isinstance(result, dict):
            best_match = result.get("string")
            score = result.get("score", 0)
        else:
            best_match, score = None, 0

        # If the score is above the threshold, consider it a match
        if score >= threshold:
            matches[lga] = best_match
        else:
            matches[lga] = None  # No match found

    # Add a new column 'matched_ADM2_EN' to df_nihsa based on the matches
    df_nihsa["matched_ADM2_EN"] = df_nihsa["LGA"].map(matches)

    # Identify LGAs that did not get matched
    unmatched = df_nihsa[df_nihsa["matched_ADM2_EN"].isna()]

    # Provide a report of unmatched entries for manual assignment
    if not unmatched.empty:
        print("The following LGAs were not automatically matched:")
        print(unmatched[["LGA"]])
        print("\nPlease manually assign matches to these LGAs.")

    # Confirm all LGA entries got matched (if you want to raise an error if not)
    if unmatched.shape[0] > 0:
        raise ValueError(
            f"{unmatched.shape[0]} LGAs were not matched. Please check the unmatched list above."
        )

    return df_nihsa


# Run the function with your dataframes
df_nihsa = match_LGAs(df_nihsa, adm2)

In [45]:
adm2[adm2["ADM1_PCODE"].isin(AOI_ADM1_PCODES)].iloc[:60]

,ADM2_EN,ADM2_PCODE,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,DATE,VALIDON,VALIDTO,geometry
2,Abadam,NG008001,Borno,NG008,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((13.11029 13.10963, 13.10815 13.11404..."
68,Askira/Uba,NG008002,Borno,NG008,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((13.09852 10.44508, 13.09352 10.44667..."
82,Bade,NG036001,Yobe,NG036,Nigeria,NG,2017-03-31,2019-04-17,None,"MULTIPOLYGON (((11.18769 12.50815, 11.18360 12..."
90,Bama,NG008003,Borno,NG008,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((13.99056 11.31251, 13.97581 11.30513..."
99,Bayo,NG008004,Borno,NG008,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((11.65688 10.19681, 11.60128 10.20806..."
114,Biu,NG008005,Borno,NG008,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((12.16747 10.59467, 12.16747 10.59467..."
131,Bursari,NG036002,Yobe,NG036,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((11.50375 12.46693, 11.49972 12.46416..."
139,Chibok,NG008006,Borno,NG008,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((12.94077 10.70080, 12.93458 10.70301..."
142,Damaturu,NG036003,Yobe,NG036,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((11.76168 11.50852, 11.75896 11.50665..."
145,Damboa,NG008007,Borno,NG008,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((12.90696 11.27313, 12.91443 11.27299..."


In [46]:
adm2[adm2["ADM1_PCODE"].isin(AOI_ADM1_PCODES)].iloc[60:]

,ADM2_EN,ADM2_PCODE,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,DATE,VALIDON,VALIDTO,geometry
709,Toungo,NG002019,Adamawa,NG002,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((12.21046 7.97042, 12.20300 7.96432, ..."
762,Yola North,NG002020,Adamawa,NG002,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((12.40447 9.25767, 12.40226 9.26307, ..."
763,Yola South,NG002021,Adamawa,NG002,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((12.43708 9.21300, 12.43732 9.21285, ..."
765,Yunusari,NG036016,Yobe,NG036,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((12.13949 13.09935, 12.14254 13.09896..."
766,Yusufari,NG036017,Yobe,NG036,Nigeria,NG,2017-03-31,2019-04-17,None,"POLYGON ((11.32417 13.15145, 11.32829 13.14117..."


In [47]:
df_nihsa = df_nihsa.rename(columns={"matched_ADM2_EN": "ADM2_EN"})

In [48]:
df_nihsa

,Year,State,LGA,Flooded,ADM2_EN
17,2013,Adamawa,Demsa,False,Demsa
18,2013,Adamawa,Fufore,False,Fufore
19,2013,Adamawa,Ganye,False,Ganye
20,2013,Adamawa,Girei,False,Girei
21,2013,Adamawa,Gombi,False,Gombi
...,...,...,...,...,...
8507,2023,Yobe,Bursari,False,Bursari
8508,2023,Yobe,Karasuwa,False,Karasuwa
8509,2023,Yobe,Yusufari,False,Yusufari
8510,2023,Yobe,Yunusari,True,Yunusari


In [49]:
df_nihsa = df_nihsa.merge(adm2[["ADM2_EN", "ADM2_PCODE"]])

In [50]:
df_nihsa

,Year,State,LGA,Flooded,ADM2_EN,ADM2_PCODE
0,2013,Adamawa,Demsa,False,Demsa,NG002001
1,2013,Adamawa,Fufore,False,Fufore,NG002002
2,2013,Adamawa,Ganye,False,Ganye,NG002003
3,2013,Adamawa,Girei,False,Girei,NG002005
4,2013,Adamawa,Gombi,False,Gombi,NG002004
...,...,...,...,...,...,...
710,2023,Yobe,Bursari,False,Bursari,NG036002
711,2023,Yobe,Karasuwa,False,Karasuwa,NG036010
712,2023,Yobe,Yusufari,False,Yusufari,NG036017
713,2023,Yobe,Yunusari,True,Yunusari,NG036016


In [51]:
blob_name = (
    f"{blob.PROJECT_PREFIX}/processed/nihsa/floodhistory_2013_2023.parquet"
)

In [52]:
blob.upload_parquet_to_blob(blob_name, df_nihsa.drop(columns=["State", "LGA"]))